In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

In [2]:
data_directory = 'data/'
image_directory = 'data/Pre_train/'

In [3]:
size = 32
batch_size = 256
num_channels = 3

In [4]:
def read_label_files(csv_file):
    file = open(data_directory+csv_file, 'r')
    filepaths = []
    labels = []
    for i in file:
        filename, label = i.split(',')
        filepaths.append(image_directory+filename)
        labels.append(int(label))
    return filepaths, labels    

In [5]:
train_path, train_labels = read_label_files('train_set.csv')
vali_path, vali_labels = read_label_files('vali_set.csv')

In [6]:
train_path_tensor = ops.convert_to_tensor(train_path, dtype=dtypes.string)
vali_path_tensor = ops.convert_to_tensor(vali_path, dtype=dtypes.string)
train_labels_tensor = ops.convert_to_tensor(train_labels, dtype=dtypes.int32)
vali_labels_tensor = ops.convert_to_tensor(vali_labels, dtype=dtypes.int32)

In [7]:
train_input_queue = tf.train.slice_input_producer(
                                    [train_path_tensor, train_labels_tensor],
                                    shuffle=True)
vali_input_queue = tf.train.slice_input_producer(
                                    [vali_path_tensor, vali_labels_tensor],
                                    shuffle=True)

In [15]:
file_content = tf.read_file(train_input_queue[0])
train_image = tf.image.decode_jpeg(file_content, channels=num_channels)
train_image /= 255
train_label = train_input_queue[1]

file_content = tf.read_file(vali_input_queue[0])
vali_image = tf.image.decode_jpeg(file_content, channels=num_channels)
vali_image /= 255
vali_label = vali_input_queue[1]

In [16]:
train_image.set_shape([size, size, num_channels])
vali_image.set_shape([size, size, num_channels])

train_batch = tf.train.batch(
                            [train_image, train_label],
                            batch_size= batch_size
                            )
vali_batch = tf.train.batch(
                            [vali_image, vali_label],
                            batch_size = batch_size
                            )

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(1):
        print(sess.run(train_batch))
    coord.request_stop()
    coord.join(threads)
    

[array([[[[ 0.25490198,  0.13725491,  0.17647059],
         [ 0.25490198,  0.13725491,  0.17647059],
         [ 0.27843139,  0.15294118,  0.19607843],
         ..., 
         [ 0.26666668,  0.16862746,  0.25490198],
         [ 0.30588236,  0.20784314,  0.29411766],
         [ 0.32549021,  0.22745098,  0.3137255 ]],

        [[ 0.24705882,  0.12941177,  0.16862746],
         [ 0.22352941,  0.10588235,  0.14509805],
         [ 0.21176471,  0.08627451,  0.12941177],
         ..., 
         [ 0.24313726,  0.14509805,  0.23137255],
         [ 0.27058825,  0.17254902,  0.25882354],
         [ 0.28235295,  0.18431373,  0.27058825]],

        [[ 0.20392157,  0.08627451,  0.13333334],
         [ 0.20392157,  0.08627451,  0.13333334],
         [ 0.21960784,  0.09411765,  0.13725491],
         ..., 
         [ 0.23529412,  0.1254902 ,  0.21176471],
         [ 0.24313726,  0.14509805,  0.23137255],
         [ 0.24313726,  0.14509805,  0.23137255]],

        ..., 
        [[ 0.1254902 ,  0.06666667